# Level 1: Rice Crop Discovery Tool Notebook

In [1]:
# Supress Warnings
import warnings
warnings.filterwarnings('ignore')

#Date time
import datetime as dt
from datetime import timedelta, date

# Visualization
import ipyleaflet
import matplotlib.pyplot as plt
from IPython.display import Image
import seaborn as sns

# Data Science
import numpy as np
import pandas as pd

# Feature Engineering
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Machine Learning
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, accuracy_score,classification_report,confusion_matrix

# Planetary Computer Tools
import pystac
import pystac_client
import odc
from pystac_client import Client
from pystac.extensions.eo import EOExtension as eo
from odc.stac import stac_load
import planetary_computer as pc
pc.settings.set_subscription_key('b0067a12405d4fd4a4cc82d28869d9bc')

# Others
import requests
import rich.table
from itertools import cycle
from tqdm import tqdm
tqdm.pandas()

#### Reading dataset provided by EY data challenge

In [2]:
crop_presence_data = pd.read_csv("./Data/Crop_Location_Data_20221201.csv")
crop_presence_data.head()

,Latitude and Longitude,Class of Land
0,"(10.323727047081501, 105.2516346045924)",Rice
1,"(10.322364360592521, 105.27843410554115)",Rice
2,"(10.321455902933202, 105.25254306225168)",Rice
3,"(10.324181275911162, 105.25118037576274)",Rice
4,"(10.324635504740822, 105.27389181724476)",Rice


In [3]:
def get_sentinel_data(latlong,time_slice,assets):
    latlong=latlong.replace('(','').replace(')','').replace(' ','').split(',')
    bbox_of_interest = (float(latlong[1]) , float(latlong[0]), float(latlong[1]) , float(latlong[0]))
    time_of_interest = time_slice
    
    catalog = pystac_client.Client.open(
        "https://planetarycomputer.microsoft.com/api/stac/v1"
    )
    search = catalog.search(
        collections=["sentinel-1-rtc"], bbox=bbox_of_interest, datetime=time_of_interest
    )
    items = list(search.get_all_items())
    bands_of_interest = assests
    data = stac_load([items[0]], patch_url=pc.sign, bbox=bbox_of_interest).isel(time=0)
    vh = data["vh"].astype("float").values.tolist()[0][0]
    vv = data["vv"].astype("float").values.tolist()[0][0]
    
    return vh,vv

In [5]:
## Function call to extract VV,VH Values
time_slice = "2020-03-20/2020-03-21"
assests = ['vh','vv']
vh_vv = []
for coordinates in tqdm(crop_presence_data['Latitude and Longitude']):
    vh_vv.append(get_sentinel_data(coordinates,time_slice,assests))
vh_vv_data = pd.DataFrame(vh_vv,columns =['vh','vv'])

100%|████████████████████████████████████████████████████████████████████████████████| 600/600 [02:38<00:00,  3.80it/s]


In [8]:
def combine_all_datasets(dataset1,dataset2, dataset3):
    '''
    Returns a  vertically concatenated dataset.
    Attributes:
    dataset1 - Dataset 1 to be combined 
    dataset2 - Dataset 2 to be combined
    dataset3- Dataset 3 to be combined
    '''
    data = pd.concat([dataset1,dataset2, dataset3], axis=1)
    return data

### RVI calculation

#### Calculate the bounding box

In [50]:
def calculate_bbox(lat_long, box_size_deg=0.0004):
    lat_long=lat_long.replace('(','').replace(')','').replace(' ','').split(',')
    
    min_lon = float(lat_long[1]) - box_size_deg/2
    min_lat = float(lat_long[0])- box_size_deg/2
    max_lon = float(lat_long[1]) + box_size_deg/2
    max_lat = float(lat_long[0]) + box_size_deg/2
    
    return min_lon, min_lat, max_lon, max_lat


In [51]:
time_of_interest = '2021-12-01/2022-12-01'
resolution = 10  # meters per pixel 
scale = resolution / 111320.0 # degrees per pixel for crs=4326 

In [57]:
def create_rvi_for_each_aoi(time_of_interest, bbox):
    catalog = pystac_client.Client.open("https://planetarycomputer.microsoft.com/api/stac/v1")
    search = catalog.search(collections=["sentinel-1-rtc"], bbox=bbox, datetime=time_of_interest)
    items = list(search.get_all_items())
    # Load the data using Open Data Cube
    data = stac_load(items,bands=["vv", "vh"], patch_url=pc.sign, bbox=bbox, crs="EPSG:4326", resolution=scale)
    # Calculate the mean of the data across the sample region
    mean = data.mean(dim=['latitude','longitude']).compute()
    # Calculate RVI
    dop = (mean.vv / (mean.vv + mean.vh))
    m = 1 - dop
    rvi = (np.sqrt(dop))*((4*mean.vh)/(mean.vv + mean.vh))
    return rvi

In [53]:
rvi_df = pd.DataFrame()
bbox = []
for coordinates in tqdm(crop_presence_data['Latitude and Longitude']):
    bbox.append(calculate_bbox(coordinates))

bbox_data = pd.DataFrame(bbox,columns =['min_lon', 'min_lat', 'max_lon', 'max_lat'])
#create_scene_id(time_of_interest, bbox)

100%|████████████████████████████████████████████████████████████████████████████| 600/600 [00:00<00:00, 101417.84it/s]


In [ ]:
for items in 
items = create_scene_id(time_of_interest, )

In [18]:
# Load the data using Open Data Cube
data = stac_load(items,bands=["vv", "vh"], patch_url=pc.sign, bbox=bbox, crs="EPSG:4326", resolution=scale)

NameError: name 'items' is not defined